In [1]:
import sys

sys.path.append("/home/td-user/rtrec")

# Movielens 1M

In [2]:
from rtrec.experiments.datasets import load_dataset

df = load_dataset(name='movielens_1m')

Using existing ratings.dat file.


In [3]:
df[['user', 'item', 'tstamp', 'rating']]

,user,item,tstamp,rating
0,1,1193,978300760,5
1,1,661,978302109,3
2,1,914,978301968,3
3,1,3408,978300275,4
4,1,2355,978824291,5
...,...,...,...,...
1000204,6040,1091,956716541,1
1000205,6040,1094,956704887,5
1000206,6040,562,956704746,5
1000207,6040,1096,956715648,4


In [4]:
# from rtrec.experiments.split import leave_one_last_item
# train_df, test_df = leave_one_last_item(df)

from rtrec.experiments.split import temporal_user_split
train_df, test_df = temporal_user_split(df)

#from rtrec.experiments.split import random_split
#train_df, test_df = random_split(df)

In [5]:
train_df

,user,item,rating,tstamp
0,6040,858,4,956703932
1,6040,593,5,956703954
2,6040,2384,4,956703954
3,6040,1961,4,956703977
4,6040,2019,5,956703977
...,...,...,...,...
797753,736,1278,4,1045711206
797754,736,3671,4,1045711217
797755,3840,1196,3,1046106127
797756,3840,1225,3,1046106162


# SLIM (Sparse Linear Methods)

In [6]:
from rtrec.recommender import Recommender
from rtrec.models import SLIM

#model = SLIM()
# nn_feature_selection is fast
model = SLIM(nn_feature_selection=50) 

recommender = Recommender(model)

In [7]:
recommender.bulk_fit(train_df)

Fitting SLIMElastic in parallel: 100%|████████████████████████████████████████████████████████████████████████| 39/39 [00:05<00:00,  7.36it/s]

Fit completed in 7.70 seconds
Throughput: 103586.26 samples/sec


In [8]:
recommender.evaluate(test_df, recommend_size=10, filter_interacted=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:06<00:00,  8.86it/s]


{'precision': 0.13187086092715705,
 'recall': 0.06753560170555596,
 'f1': 0.07295727145848262,
 'ndcg': 0.14443232260780747,
 'hit_rate': 0.6349337748344371,
 'mrr': 0.2873341085882459,
 'map': 0.0689205770424432,
 'tp': 7965,
 'auc': 0.3372786581520036}

In [9]:
recommender.model._recommend(10, top_k=10, filter_interacted=True)

[1136, 595, 260, 1394, 1035, 2797, 1304, 2406, 1259, 34]

# test partial_fit

In [10]:
from rtrec.experiments.split import temporal_split

test_df1, test_df2 = temporal_split(test_df, test_frac=0.5)

In [11]:
from rtrec.recommender import Recommender
from rtrec.models import SLIM

model = SLIM(nn_feature_selection=50)
recommender = Recommender(model)

In [12]:
recommender.bulk_fit(train_df)

Fitting SLIMElastic in parallel: 100%|████████████████████████████████████████████████████████████████████████| 39/39 [00:05<00:00,  7.66it/s]

Fit completed in 7.42 seconds
Throughput: 107458.90 samples/sec


### Note test_df1 is not fit

In [13]:
recommender.evaluate(test_df2, recommend_size=10, filter_interacted=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:03<00:00,  8.39it/s]


{'precision': 0.1423777222420567,
 'recall': 0.06746896811273603,
 'f1': 0.07364132623888481,
 'ndcg': 0.15607206466519727,
 'hit_rate': 0.6511960014280614,
 'mrr': 0.30404773238582605,
 'map': 0.07677367643012346,
 'tp': 3988,
 'auc': 0.3515945977343692}

### Fit test_df1

In [14]:
recommender.fit(test_df1, update_interaction=True, parallel=True)

Fitting SLIMElastic in parallel: 100%|████████████████████████████████████████████████████████████████████████| 39/39 [00:04<00:00,  8.07it/s]

Fit completed in 20.32 seconds
Throughput: 4980.58 samples/sec


In [15]:
recommender.evaluate(test_df2, recommend_size=10, filter_interacted=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:03<00:00,  8.21it/s]


{'precision': 0.14623348803998612,
 'recall': 0.07076125439505432,
 'f1': 0.07699598570870538,
 'ndcg': 0.1611090192445629,
 'hit_rate': 0.6636915387361657,
 'mrr': 0.3154465808696444,
 'map': 0.07936593805941061,
 'tp': 4096,
 'auc': 0.36074509953928036}

# LightFM (Factorization Machines)

In [51]:
from rtrec.models import LightFM

model = LightFM(no_components=10, loss="warp", epochs=20, item_alpha=0, user_alpha=0)
recommender = Recommender(model)

In [52]:
recommender.bulk_fit(train_df)

Add interactions: 100%|████████████████████████████████████████████████████████████████████████████████████| 798/798 [00:02<00:00, 387.24it/s]


Fit completed in 10.43 seconds
Throughput: 76477.35 samples/sec


In [53]:
recommender.evaluate(test_df, recommend_size=10, filter_interacted=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:00<00:00, 67.64it/s]


{'precision': 0.04360927152317792,
 'recall': 0.02341324911270315,
 'f1': 0.025827257137960326,
 'ndcg': 0.04540425122885596,
 'hit_rate': 0.3021523178807947,
 'mrr': 0.10396569168506267,
 'map': 0.01669777306618011,
 'tp': 2634,
 'auc': 0.14738366577315234}

In [60]:
recommender.model.recommend(1, top_k=10, filter_interacted=False)

[2858, 1210, 260, 480, 1196, 2028, 110, 1580, 593, 1617]

In [61]:
recommender.model.recommend(1, top_k=10, filter_interacted=True)

[2858, 1210, 480, 1196, 110, 1580, 593, 1617, 858, 589]

In [66]:
recommender.model.recommend_batch([1], top_k=10, filter_interacted=False)

[[2858, 1210, 260, 480, 1196, 2028, 110, 1580, 593, 1617]]

In [57]:
recommender.model._recommend(2, top_k=10, filter_interacted=False)

[2858, 1210, 260, 480, 1196, 2028, 110, 1580, 593, 1617]

In [58]:
recommender.model._recommend(2, top_k=10, filter_interacted=True)

[260, 1580, 1617, 1270, 858, 527, 1197, 2997, 2762, 2628]

In [59]:
user1_items = train_df[train_df['user']==2]['item'].tolist()
user1_items.sort()

user1_items

[21,
 110,
 163,
 235,
 265,
 318,
 349,
 356,
 380,
 457,
 480,
 498,
 515,
 589,
 590,
 593,
 647,
 648,
 902,
 920,
 982,
 1084,
 1090,
 1096,
 1103,
 1124,
 1188,
 1193,
 1196,
 1198,
 1207,
 1210,
 1213,
 1217,
 1225,
 1244,
 1245,
 1246,
 1247,
 1253,
 1259,
 1265,
 1293,
 1357,
 1370,
 1408,
 1442,
 1527,
 1537,
 1610,
 1784,
 1834,
 1873,
 1945,
 1953,
 1954,
 1955,
 1957,
 1962,
 1968,
 2006,
 2028,
 2067,
 2194,
 2236,
 2268,
 2278,
 2312,
 2321,
 2353,
 2359,
 2396,
 2427,
 2501,
 2571,
 2717,
 2728,
 2852,
 2858,
 2916,
 2943,
 3030,
 3035,
 3068,
 3071,
 3095,
 3105,
 3108,
 3147,
 3255,
 3256,
 3334,
 3418,
 3451,
 3468,
 3471,
 3578,
 3654,
 3678,
 3699,
 3735,
 3809,
 3893]